In [ ]:
#Util Functions
import time
import requests
from urllib.parse import urlparse
import sys
import json
import lxml.html
import csv

REQUEST_DELAY = 0.5


def make_request(url):
    """
    Make a request to `url` and return the raw response.

    This function ensure that the domain matches what is expected and that the rate limit
    is obeyed.
    """
    # check if URL starts with an allowed domain name
    time.sleep(REQUEST_DELAY)
    print(f"Fetching {url}")
    resp = requests.get(url)
    return resp

def parse_html(html):
    """
    Parse HTML and return the root node.
    """
    return lxml.html.fromstring(html)

def make_link_absolute(rel_url, current_url):
    """
    Given a relative URL like "/abc/def" or "?page=2"
    and a complete URL like "https://example.com/1/2/3" this function will
    combine the two yielding a URL like "https://example.com/abc/def"

    Parameters:
        * rel_url:      a URL or fragment
        * current_url:  a complete URL used to make the request that contained a link to rel_url

    Returns:
        A full URL with protocol & domain that refers to rel_url.
    """
    url = urlparse(current_url)
    if rel_url.startswith("/"):
        return f"{url.scheme}://{url.netloc}{rel_url}"
    elif rel_url.startswith("?"):
        return f"{url.scheme}://{url.netloc}{url.path}{rel_url}"
    else:
        return rel_url

: 

In [ ]:
def page_grab(url):
    urls = []
    response = make_request(url)
    root = parse_html(response.text)
    return (root ,url)


: 

In [ ]:
#Institute for Fox News

def crawl_fox(base_page, article=set(),video=set()):
    response= page_grab(base_page)
    json_data = json.loads(response[0].text)
    json_data
    for i in json_data:
        url= make_link_absolute(i['url'], "https://www.foxnews.com/politics")
        if url.startswith("https://www.foxnews.com/politics"):
            article.add(url)
        else:
            video.add(url)
    begin=base_page.find("from")+5
    end=base_page.find("&media")
    articlenumber= int(base_page[begin:end])
    if articlenumber <9970:
        articlenumber+= 30
        articlenumber=str(articlenumber)
        rev_basepage=base_page[0:begin]+articlenumber+base_page[end:(len(base_page)+1)]
        crawl_fox(rev_basepage, article,video)
    return article, video



article, video = crawl_fox("https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=15&mediaTags=primary_politics")

article, video



: 

In [ ]:
#save data via json
json_data = (json.dumps(list(article)))
json_data

#save as csv
def create_csv(set1, set2, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Articles', 'Video'])
        max_length = max(len(set1), len(set2))
        for i in range(max_length):
            row = [list(set1)[i] if i < len(set1) else '', list(set2)[i] if i < len(set2) else '']
            writer.writerow(row)

# Calling the function to create the CSV file
create_csv(article, video, 'fox_articles.csv')



: 